<a href="https://colab.research.google.com/github/Aneesh-Pal/Aneesh-PaL/blob/main/Aneeshpal_Cirrhosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 29 kB/s 
     |████████████████████████████████| 198 kB 52.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=aae382a56800bd6c99c63d030a3fef3f7e7b63fb2f7dd03f2f4cae32f525582e
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
session=SparkSession.builder.appName("MyCode").getOrCreate()

In [4]:
df=session.read.csv("cirrhosis.csv",header=True, inferSchema=True)

In [6]:
df.show(5)

+---+------+------+---------------+-----+---+-------+------------+-------+-----+---------+-----------+-------+------+--------+------+-------------+---------+-----------+-----+
| ID|N_Days|Status|           Drug|  Age|Sex|Ascites|Hepatomegaly|Spiders|Edema|Bilirubin|Cholesterol|Albumin|Copper|Alk_Phos|  SGOT|Tryglicerides|Platelets|Prothrombin|Stage|
+---+------+------+---------------+-----+---+-------+------------+-------+-----+---------+-----------+-------+------+--------+------+-------------+---------+-----------+-----+
|  1|   400|     D|D-penicillamine|21464|  F|      Y|           Y|      Y|    Y|     14.5|        261|    2.6|   156|    1718|137.95|          172|      190|       12.2|    4|
|  2|  4500|     C|D-penicillamine|20617|  F|      N|           Y|      Y|    N|      1.1|        302|   4.14|    54|  7394.8|113.52|           88|      221|       10.6|    3|
|  3|  1012|     D|D-penicillamine|25594|  M|      N|           N|      N|    S|      1.4|        176|   3.48|   210|   

In [7]:
df.columns

['ID',
 'N_Days',
 'Status',
 'Drug',
 'Age',
 'Sex',
 'Ascites',
 'Hepatomegaly',
 'Spiders',
 'Edema',
 'Bilirubin',
 'Cholesterol',
 'Albumin',
 'Copper',
 'Alk_Phos',
 'SGOT',
 'Tryglicerides',
 'Platelets',
 'Prothrombin',
 'Stage']

In [9]:
df.describe

<bound method DataFrame.describe of DataFrame[ID: int, N_Days: int, Status: string, Drug: string, Age: int, Sex: string, Ascites: string, Hepatomegaly: string, Spiders: string, Edema: string, Bilirubin: double, Cholesterol: string, Albumin: double, Copper: string, Alk_Phos: string, SGOT: string, Tryglicerides: string, Platelets: string, Prothrombin: string, Stage: string]>

In [8]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder

In [61]:
str_idx=StringIndexer(inputCols=['Status','Drug','Sex','Ascites','Hepatomegaly','Spiders','Edema','Bilirubin','Albumin','Cholesterol','Alk_Phos','SGOT','Tryglicerides','Platelets','Prothrombin','Copper','Stage'],outputCols=['new_Status','new_Drug','new_Sex','new_Ascites','new_Hepatomegaly','new_Spiders','new_Edema','new_Bilirubin','new_Albumin','new_Cholesterol','new_Alk_Phos','new_SGOT','new_Tryglicerides','new_Platelets','new_Prothrombin','new_Copper','new_Stage'],handleInvalid="skip")

In [62]:
onehot=OneHotEncoder(inputCols=['new_Status','new_Drug','new_Sex','new_Ascites','new_Hepatomegaly','new_Spiders','new_Edema'],outputCols=['new_Status2','new_Drug2','new_Sex2','new_Ascites2','new_Hepatomegaly2','new_Spiders2','new_Edema2'])

In [63]:
vect=VectorAssembler(inputCols=['N_Days','new_Status2','new_Drug2','Age','new_Sex2','new_Ascites2','new_Hepatomegaly2','new_Spiders2','new_Edema2','new_Bilirubin','new_Cholesterol','new_Albumin','new_Copper','new_Alk_Phos','new_SGOT','new_Tryglicerides','new_Platelets','new_Prothrombin'],outputCol="allfeatures")

In [68]:
from pyspark.ml.classification import LogisticRegression
log_reg = LogisticRegression(featuresCol="allfeatures", labelCol = "new_Stage")

In [69]:
from pyspark.ml import Pipeline
mypipeline = Pipeline(stages = [str_idx, onehot, vect, log_reg])

In [70]:
training, test = df.randomSplit([0.8, 0.2])

In [71]:
log_reg_model = mypipeline.fit(training)

In [72]:
result = log_reg_model.transform(test)

In [73]:
result.show()

+---+------+------+---------------+-----+---+-------+------------+-------+-----+---------+-----------+-------+------+--------+------+-------------+---------+-----------+-----+----------+--------+-------+-----------+----------------+-----------+---------+-------------+-----------+---------------+------------+--------+-----------------+-------------+---------------+----------+---------+-------------+-------------+-------------+-------------+-----------------+-------------+-------------+--------------------+--------------------+--------------------+----------+
| ID|N_Days|Status|           Drug|  Age|Sex|Ascites|Hepatomegaly|Spiders|Edema|Bilirubin|Cholesterol|Albumin|Copper|Alk_Phos|  SGOT|Tryglicerides|Platelets|Prothrombin|Stage|new_Status|new_Drug|new_Sex|new_Ascites|new_Hepatomegaly|new_Spiders|new_Edema|new_Bilirubin|new_Albumin|new_Cholesterol|new_Alk_Phos|new_SGOT|new_Tryglicerides|new_Platelets|new_Prothrombin|new_Copper|new_Stage|  new_Status2|    new_Drug2|     new_Sex2| new_As

In [76]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
eval = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol = "new_Stage")

In [77]:
eval.evaluate(result)

0.5875